In [2]:
import sys
sys.path.append("..")
import pickle
import numpy as np
import pandas as pd
import sys
from scipy import optimize
from torch.utils.data import DataLoader, Dataset
from data_loaders import *
import missing_process.missing_method as missing_method
from missing_process.block_rules import *
import json
import matplotlib.pyplot as plt


In [3]:
real_datalist = ["banknote","concrete_compression",
            "wine_quality_white","wine_quality_red",
            "california","climate_model_crashes",
            "connectionist_bench_sonar","qsar_biodegradation",
            "yeast","yacht_hydrodynamics"
            ]

syn_datalist = ["syn1"]


missingtypelist = ["mcar","mar","quantile","diffuse","logistic"]

In [ ]:
def create_mask(observed_values,missing_type = "MCAR",
                  missing_para = None):

    observed_masks = ~np.isnan(observed_values.astype("float32"))
    masks = observed_masks.copy().astype("float32")


    "Need input origin dataset and parameters"
    if missing_type == "mcar":
        masks = missing_method.MCAR(observed_values,missing_para)


    elif missing_type == "quantile":
        Xnan, Xz = missing_method.missing_by_range(observed_values, missing_para)
        masks = np.array(~np.isnan(Xnan), dtype=np.float)


    elif missing_type == "logistic":
        masks = missing_method.MNAR_mask_logistic(observed_values, missing_para)


    elif missing_type == "diffuse":
        masks =  missing_method.diffuse_mnar_single(observed_values, missing_para[0],missing_para[1])

    elif missing_type == "mar":

        masks = missing_method.MAR_mask(observed_values,1,missing_para)

        

    observed_masks = masks.astype(int)
    return observed_masks


In [7]:
def create_missing(missingtypelist,datalist):
    '''
    Create Mask npy
    Save missing 
    '''

    for missingtype in missingtypelist:
        if missingtype == "logistic":
            missing_rule = load_json_file("missing_rate.json")
        elif missingtype == "diffuse":
            missing_rule = load_json_file("diffuse_ratio.json")
        elif missingtype == "quantile":
            missing_rule = load_json_file("quantile_full.json")

        elif missingtype == "mcar" or missingtype == "mar":
            missing_rule = load_json_file("mcar.json")


        missing_rate_d = {}

        for dataname in datalist:
            directory_path = f"../datasets/{dataname}"    
            norm_values = np.load(f'{directory_path}/{dataname}_norm.npy')

            print(dataname)
            missing_rate_d[dataname] = []

                # Check if the directory exists
            if not os.path.exists(f'{directory_path}/{missingtype}'):
                os.makedirs(f'{directory_path}/{missingtype}')
            else:
                pass
            
            for rule_name in missing_rule:

                rule = missing_rule[rule_name]
                # try:
                #     create_mask(norm_values,missingtype,rule)
                # except:
                #     print(dataname,missingtype)
                
                observed_masks = create_mask(norm_values,missingtype,rule)

                np.save(f'{directory_path}/{missingtype}/{rule_name}.npy', observed_masks)



In [ ]:
create_missing(missingtypelist,real_datalist)